In [83]:
#coding:utf-8

In [84]:
import tensorflow as tf
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, LSTM
from tensorflow.keras.layers import TextVectorization

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
data = pd.read_csv("/content/drive/MyDrive/EE深度學習/name,outline,totalFundedUSD.csv")
data = data.drop('country_code', axis=1)
data = data.drop('funding_rounds', axis=1)
data = data.drop('status', axis=1)
data = data.drop('entity_id', axis=1)
data = data.drop('normalized_name', axis=1)
data = data.dropna()
data['funding_total_usd'] = data['funding_total_usd'].apply(lambda x: x / data['funding_total_usd'].max())
print(data)
trainfundingTotalUsd = data['funding_total_usd'].to_numpy()
trainOverviewString = data['overview'].to_numpy()
print(trainfundingTotalUsd.shape)
print(trainOverviewString.shape)


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


                                                 overview  funding_total_usd
0       Wetpaint is a technology platform company that...           0.006974
1       Flektor is a rich-media mash-up platform that ...           0.000000
2       There.com is an online virtual world where any...           0.000000
3       BRAND NEW ONLINE SOCIAL NETWORKING WEBSITE,FOR...           0.000000
4       This company shows free movies online on their...           0.000000
...                                                   ...                ...
462629   Get the latest news from around the world and...           0.000000
462634  Pintarget helps people to research online.  \n...           0.000000
462637  The Nutrition Advisors' Network?¢â???¢, is pow...           0.000000
462642  Advertisers, do you know if your banners are a...           0.000000
462643  Measure the real visibility of your ad-invento...           0.000000

[235235 rows x 2 columns]
(235235,)
(235235,)


In [86]:
tokenizer = Tokenizer(num_words=1000, lower=True)
tokenizer.fit_on_texts(trainOverviewString)

In [87]:
trainOverviewString = tokenizer.texts_to_sequences(trainOverviewString)

In [88]:
trainOverviewString = pad_sequences(trainOverviewString, padding='post', maxlen=1000)
print(trainOverviewString.shape)

(235235, 1000)


In [89]:
model = Sequential([
  Embedding(input_dim=1000, output_dim=16),
  LSTM(128),
  Dense(16, activation='relu'),
  Dense(1, activation='softmax')
])

In [90]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 16)          16000     
                                                                 
 lstm_6 (LSTM)               (None, 128)               74240     
                                                                 
 dense_12 (Dense)            (None, 16)                2064      
                                                                 
 dense_13 (Dense)            (None, 1)                 17        
                                                                 
Total params: 92,321
Trainable params: 92,321
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [92]:
print(trainOverviewString.shape)
print(trainfundingTotalUsd.shape)
train_history = model.fit(trainOverviewString, trainfundingTotalUsd, validation_split=0.2, epochs=30, batch_size=100, verbose=1)

(235235, 1000)
(235235,)
Epoch 1/30
  47/1882 [..............................] - ETA: 1:18:17 - loss: 0.9991 - accuracy: 0.0000e+00

KeyboardInterrupt: ignored